We will work on this dataset on jupyter notebooks:
dataset path : data/yellow_tripdata_2023-01.parquet
Data Dictionary – Yellow Taxi Trip Records May 11, 2022 Page 1 of 2
This data dictionary describes yellow taxi trip data. For a dictionary describing green taxi data, or a map
of the TLC Taxi Zones, please visit http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml.
Field Name Description
VendorID A code indicating the TPEP provider that provided the record.
1= Creative Mobile Technologies, LLC; 2= VeriFone Inc.
tpep_pickup_datetime The date and time when the meter was engaged.
tpep_dropoff_datetime The date and time when the meter was disengaged.
Passenger_count The number of passengers in the vehicle.
This is a driver-entered value.
Trip_distance The elapsed trip distance in miles reported by the taximeter.
PULocationID TLC Taxi Zone in which the taximeter was engaged
DOLocationID TLC Taxi Zone in which the taximeter was disengaged
RateCodeID The final rate code in effect at the end of the trip.
1= Standard rate
2=JFK
3=Newark
4=Nassau or Westchester
5=Negotiated fare
6=Group ride
Store_and_fwd_flag This flag indicates whether the trip record was held in vehicle
memory before sending to the vendor, aka “store and forward,”
because the vehicle did not have a connection to the server.
Y= store and forward trip
N= not a store and forward trip
Payment_type A numeric code signifying how the passenger paid for the trip.
1= Credit card
2= Cash
3= No charge
4= Dispute
5= Unknown
6= Voided trip
Fare_amount The time-and-distance fare calculated by the meter.
Extra Miscellaneous extras and surcharges. Currently, this only includes
the $0.50 and $1 rush hour and overnight charges.
MTA_tax $0.50 MTA tax that is automatically triggered based on the metered
rate in use.
Improvement_surcharge $0.30 improvement surcharge assessed trips at the flag drop. The
improvement surcharge began being levied in 2015.
Tip_amount Tip amount – This field is automatically populated for credit card
tips. Cash tips are not included.
Tolls_amount Total amount of all tolls paid in trip.
Total_amount The total amount charged to passengers. Does not include cash tips.
Congestion_Surcharge Total amount collected in trip for NYS congestion surcharge.
Airport_fee $1.25 for pick up only at LaGuardia and John F. Kennedy Airports
Data Dictionary – Yellow Taxi Trip Records May 11, 2022 Page 2 of 2

In [1]:
import pandas as pd
import numpy as np

# Read the parquet file
df = pd.read_parquet('data/yellow_tripdata_2023-01.parquet')

# Display basic information about the dataset
print(df.info())

# Check for missing values
print(df.isnull().sum())

# Remove rows with missing values
df = df.dropna()

# Convert datetime columns to datetime type
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

# Calculate trip duration in minutes
df['trip_duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

# Remove trips with negative duration or unreasonably long duration (e.g., more than 24 hours)
df = df[(df['trip_duration'] > 0) & (df['trip_duration'] <= 1440)]

# Remove trips with zero passengers or unreasonably high passenger count (e.g., more than 8)
df = df[(df['passenger_count'] > 0) & (df['passenger_count'] <= 8)]

# Remove trips with zero or negative fare amount
df = df[df['fare_amount'] > 0]

# Remove trips with unreasonably long distances (e.g., more than 100 miles)
df = df[df['trip_distance'] <= 100]

# Convert categorical columns to appropriate data types
categorical_columns = ['VendorID', 'store_and_fwd_flag', 'payment_type', 'PULocationID', 'DOLocationID']
# Check if 'RateCodeID' exists in the dataframe
if 'RateCodeID' in df.columns:
    categorical_columns.append('RateCodeID')
elif 'rate_code_id' in df.columns:
    categorical_columns.append('rate_code_id')

for col in categorical_columns:
    if col in df.columns:
        df[col] = df[col].astype('category')

# Display summary statistics of the cleaned dataset
print(df.describe())

# Save the cleaned dataset
df.to_parquet('data/cleaned_yellow_tripdata_2023-01.parquet', index=False)

print("Data cleaning completed. Cleaned dataset saved as 'cleaned_yellow_tripdata_2023-01.parquet'")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

KeyError: 'RateCodeID'